In [5]:
import json
import ast
import neal
import dimod
from dimod import Binary, Real
import numpy as np
import pandas as pd
from dimod import ConstrainedQuadraticModel, ExactCQMSolver
from dwave.system import LeapHybridCQMSampler
from itertools import chain
import copy
import copy
import time
from pulp import LpVariable, LpProblem, lpSum, LpMaximize, LpMinimize, LpStatus, listSolvers, getSolver
import warnings

warnings.filterwarnings('ignore')

In [ ]:
num_data_points = 20

In [62]:
# create constraints values for sizes, based on different limits
def constraints_func(dl, gs):
    dl_limits, cons = [], []
    c = 0
    dl.reverse()
    for i in dl:
        c = c + i
        dl_limits.append(c)
        if c > gs:
            cons.append(gs)
        else:
            cons.append(c)
    return cons, dl_limits


# measure sum total of priority,size etc  
def total(sizes, result):
    sum_size = []
    for i in result:
        sum_size.append(sizes[i])

    return sum(sum_size)


# data manupilation
def correct(lst):
    for i in range(int(len(lst) / 2)):
        lst = add_negative_pairs(lst)
    return lst


# takes series for targets and gs, returns target list and gs list well seprated with its limits
def separate_gs_and_fix_non_gs_targets(s):
    if s[-1] > 0:
        t = []
        for element in reversed(s):
            if element < 0:
                break
            t.append(element)

        s = t[::-1] + s

    s = correct(s)
    s_, temp, gs_ = [], [], []
    for i in s:
        if i < 0:
            gs_.append(i)
            s_.append(temp)
            temp = []
        else:
            temp.append(i)

    if not s_[0]:
        s_ = s_[1:]
        gs_ = gs_[1:]

    return s_, gs_


def is_equal_or_less(list1, list2):
    return all(x >= y for x, y in zip(list1, list2))


# post processing and mapping the results in quadrents and lower level checks on storage and energy
def post_excel(res, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_, 
         q_targets, sizes, energy_limit_, a_time, result_col, priority_col, time_col, conditions_satisfied_col,
        unsatisfied_conditions_col):
    parsed_results = []
    total_size = 0
    priority_results = 0
    total_energy = 0
    conditions_satisfied = True
    unsatisfied_conditions = []
    for j in res:
        for i in converted_dict.items():
            if j in i[1]:
                parsed_results.append(i[0])
                # print(size_dict[i[0]])
                total_size = total_size + size_dict[i[0]]
                priority_results = priority_results + priorities_dict[i[0]]
                total_energy = total_energy + energies_dict[i[0]]

    i_, isize, iq, ienergy, sum_s, sum_e, sum_size, sum_energy = [], [], [], [], [], [], [], []
    for i in target_:
        j_, jsize, jq, je, um_s, um_e = [], [], [], [], [], []
        for j in i:
            k_, ksize, kq, ke = [], [], [], []
            for k in j:
                if k in res:
                    k_.append(k)
                    kq.append(q_targets[k])
                    ksize.append(sizes[k])
                    ke.append(energy[k])
            j_.append(k_)
            jq.append(kq)
            jsize.append(ksize)
            je.append(ke)
            um_s.append(sum(ksize))
            um_e.append(sum(ke))

        i_.append(j_)
        iq.append(jq)
        isize.append(jsize)
        ienergy.append(je)
        sum_s.append(um_s)
        sum_e.append(um_e)
        sum_size.append(sum(um_s))
        sum_energy.append(sum(um_e))

    print("result", iq)
    
    sheet.write(row, result_col, str(list(chain(*list(chain(*iq))))))


    gs_list = [gs1_, gs2_, gs3_]
    for i, j, k in zip(gs_list, sum_s, sum_size):
        if k > (-1 * sum(i)):
            print("Storage total is beyond", k, -1 * sum(i))
        if j[-1] > (-1 * i[-1]):
            print("Storage not satisfied", j[-1], -1 * i[-1])
            conditions_satisfied = False
            if "Storage limit not satisfied" not in unsatisfied_conditions:
                unsatisfied_conditions.append("Storage limit not satisfied")


    if not is_equal_or_less(energy_limit_, sum_energy):
        print("Energy not satisfied")
        conditions_satisfied = False
        if "Energy limit not satisfied" not in unsatisfied_conditions:
            unsatisfied_conditions.append("Energy limit not satisfied")

    print("Total priority:", priority_results)
    sheet.write(row, priority_col, int(priority_results))
    print(f"exceution time :{a_time} ms")
    sheet.write(row, time_col, str(a_time) + " ms")
    sheet.write(row, conditions_satisfied_col, str(conditions_satisfied))
    sheet.write(row, unsatisfied_conditions_col, str(unsatisfied_conditions))
    

def post_json(res, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_,
         q_targets, sizes, energy, energy_limit_, a_time):
    parsed_results = []
    total_size = 0
    priority_results = 0
    total_energy = 0
    conditions_satisfied = True
    unsatisfied_conditions = []
    for j in res:
        for i in converted_dict.items():
            if j in i[1]:
                parsed_results.append(i[0])
                # print(size_dict[i[0]])
                total_size = total_size + size_dict[i[0]]
                priority_results = priority_results + priorities_dict[i[0]]
                total_energy = total_energy + energies_dict[i[0]]

    i_, isize, iq, ienergy, sum_s, sum_e, sum_size, sum_energy = [], [], [], [], [], [], [], []
    for i in target_:
        j_, jsize, jq, je, um_s, um_e = [], [], [], [], [], []
        for j in i:
            k_, ksize, kq, ke = [], [], [], []
            for k in j:
                if k in res:
                    k_.append(k)
                    kq.append(q_targets[k])
                    ksize.append(sizes[k])
                    ke.append(energy[k])
            j_.append(k_)
            jq.append(kq)
            jsize.append(ksize)
            je.append(ke)
            um_s.append(sum(ksize))
            um_e.append(sum(ke))

        i_.append(j_)
        iq.append(jq)
        isize.append(jsize)
        ienergy.append(je)
        sum_s.append(um_s)
        sum_e.append(um_e)
        sum_size.append(sum(um_s))
        sum_energy.append(sum(um_e))
        
    print('iq***** ', iq)
    satellite_1 = []
    satellite_2 = []
    satellite_3 = []
    satellite_row = 0
    for first_d in iq:
        if satellite_row == 0:
            satellite_1 = list(chain(*first_d))
        elif satellite_row == 1:
            satellite_2 = list(chain(*first_d))
        elif satellite_row == 2:
            satellite_3 = list(chain(*first_d))
        satellite_row = satellite_row + 1
    

    gs_list = [gs1_, gs2_, gs3_]
    for i, j, k in zip(gs_list, sum_s, sum_size):
        if k > (-1 * sum(i)):
            print("Storage total is beyond", k, -1 * sum(i))
        if j[-1] > (-1 * i[-1]):
            conditions_satisfied = False
            if "Storage limit not satisfied" not in unsatisfied_conditions:
                unsatisfied_conditions.append("Storage limit not satisfied")

    if not is_equal_or_less(energy_limit_, sum_energy):
        conditions_satisfied = False
        if "Energy limit not satisfied" not in unsatisfied_conditions:
            unsatisfied_conditions.append("Energy limit not satisfied")

    output_json = {"Result": str(list(chain(*list(chain(*iq))))), "Total Priority": priority_results,
                   "Execution Time": float(a_time), "Satisfied all Constraints?": conditions_satisfied,
                   "Constraints Not Satisfied": unsatisfied_conditions, "Targets for Satellite 1": str(satellite_1), 
                   "Targets for Satellite 2": str(satellite_2), "Targets for Satellite 3": str(satellite_3) }
    return output_json

In [7]:
def add_negative_pairs(lst):
    # base case: if the list is empty, return an empty list
    if not lst:
        return []

    # if the first element is positive, keep it and recurse on the rest of the list
    if lst[0] >= 0:
        return [lst[0]] + add_negative_pairs(lst[1:])

    # if the first two elements are negative, add them and recurse on the rest of the list
    if len(lst) > 1 and lst[1] < 0:
        return [lst[0] + lst[1]] + add_negative_pairs(lst[2:])

    # if the first element is negative and the second is positive, keep the negative element and recurse on the rest of the list
    return [lst[0]] + add_negative_pairs(lst[1:])

In [3]:
def convert_gs_to_gs_limit(satellite, ground_stations, gs_limit):
    satellite_coverage = []
    for sat in satellite:
        if isinstance(sat, int):
            satellite_coverage.append(sat)
        else:
            gs_index = ground_stations.index(int(sat.replace('GS', '')))               
            satellite_coverage.append(gs_limit[gs_index])
    return satellite_coverage

In [ ]:
def write_satellite_coverage_area(row, col, satellite):
    satellite_coverage = []
    for sat in satellite:
        if sat < 0:
            gs_limit_index = gs_limit.index(sat)
            satellite_coverage.append('GS' + str(ground_stations[gs_limit_index]))
        else:
            satellite_coverage.append(sat)
    sheet.write(row, col, str(satellite_coverage))

In [ ]:
excel_file = 'data/data-eos-20.xlsx'
sheet = book.add_worksheet('sheet1')

content = ["Targets", "Priorities", "Size of Targets", "Energies of Targets",
           "Storage Limit of Satellites","Energy Limit of Satellites",
           "Ground Stations","Limit of Ground Stations","Satellite 1", 
           "Satellite 2", "Satellite 3"]

sheet.write(0, 0, content[0])
sheet.write(0, 1, content[1])
sheet.write(0, 2, content[2])
sheet.write(0, 3, content[3])
sheet.write(0, 4, content[4])
sheet.write(0, 5, content[5])
sheet.write(0, 6, content[6])
sheet.write(0, 7, content[7])
sheet.write(0, 8, content[8])
sheet.write(0, 9, content[9])
sheet.write(0, 10, content[10])

In [ ]:
row = 0;
def write_to_excel(main_index):
    row = main_index + 1        
    sheet.write(row, 0, str(['T' + str(target) for target in targets]))
    sheet.write(row, 1, str(priorities))
    sheet.write(row, 2, str(sizes))
    sheet.write(row, 3, str(energies))
    sheet.write(row, 4, str(storage_limit))
    sheet.write(row, 5, str(energy_limit))
    sheet.write(row, 6, str(['GS' + str(ground_station) for ground_station in ground_stations]))
    sheet.write(row, 7, str([-limit for limit in gs_limit]))
    write_satellite_coverage_area(row, 8, s1)
    write_satellite_coverage_area(row, 9, s2)
    write_satellite_coverage_area(row, 10, s3)   
    

In [8]:
excel_file = 'data/data-eos-20.xlsx'
eos_data_df = pd.read_excel(excel_file)

In [9]:
eos_data_df.head()

,Targets,Priorities,Size of Targets,Energies of Targets,Storage Limit of Satellites,Energy Limit of Satellites,Ground Stations,Limit of Ground Stations,Satellite 1,Satellite 2,Satellite 3
0,"['T18', 'T43', 'T10', 'T26', 'T36', 'T11', 'T4...","[30, 3, 9, 26, 14, 10, 29, 27, 2, 11, 24, 18, ...","[36, 20, 28, 14, 15, 16, 24, 12, 26, 18, 9, 22...","[26, 10, 23, 19, 9, 4, 14, 6, 11, 25, 1, 8, 2,...","[296, 318, 391]","[100, 151, 169]","['GS3', 'GS2', 'GS1']","[37, 38, 39]","[27, 25, 'GS1', 'GS2', 10, 'GS3', 37, 15, 35, ...","[40, 24, 30, 21, 43, 23, 11, 13, 32, 38, 37, 4...","[30, 'GS2', 20, 34, 14, 27, 41, 35, 42, 17, 10..."
1,"['T43', 'T42', 'T81', 'T27', 'T16', 'T83', 'T5...","[50, 31, 16, 38, 7, 1, 9, 58, 35, 18, 4, 40, 6...","[40, 47, 36, 17, 28, 69, 71, 27, 58, 46, 12, 6...","[52, 18, 72, 4, 66, 40, 32, 49, 54, 23, 12, 21...","[314, 331, 353]","[147, 207, 255]","['GS6', 'GS1', 'GS3', 'GS5', 'GS2', 'GS4', 'GS7']","[40, 36, 36, 31, 36, 34, 32]","[28, 'GS7', 31, 60, 29, 82, 59, 45, 'GS1', 56,...","[15, 16, 'GS6', 'GS7', 80, 59, 84, 70, 75, 51,...","[84, 78, 37, 25, 17, 39, 14, 42, 18, 83, 70, 7..."
2,"['T11', 'T25', 'T26', 'T31', 'T36', 'T10', 'T1...","[1, 20, 17, 26, 12, 5, 9, 6, 18, 2, 7, 24, 22,...","[6, 10, 4, 23, 14, 5, 16, 20, 13, 27, 15, 11, ...","[15, 4, 9, 25, 2, 24, 7, 10, 12, 20, 14, 3, 19...","[284, 309, 351]","[153, 144, 266]","['GS3', 'GS2', 'GS1']","[31, 33, 34]","[29, 'GS1', 24, 23, 27, 33, 30, 19, 32, 17, 10...","[22, 36, 34, 33, 'GS2', 21, 27, 14, 19, 16, 10...","[13, 'GS3', 'GS1', 'GS2', 14]"
3,"['T95', 'T37', 'T13', 'T16', 'T77', 'T97', 'T1...","[29, 85, 44, 2, 42, 91, 25, 82, 23, 18, 24, 73...","[16, 74, 28, 95, 94, 20, 65, 33, 9, 15, 40, 14...","[21, 42, 26, 47, 51, 86, 72, 87, 93, 23, 74, 5...","[321, 329, 288]","[223, 238, 133]","['GS8', 'GS5', 'GS6', 'GS3', 'GS2', 'GS7', 'GS...","[30, 34, 35, 32, 37, 30, 33, 35]","[42, 75, 81, 29, 'GS5', 'GS2', 63, 87, 62, 94,...","[101, 69, 72, 50, 'GS3', 89, 'GS6', 68, 91, 54...","[33, 31, 39, 80, 41, 36, 42, 'GS5', 15, 88, 94..."
4,"['T10', 'T19', 'T13', 'T15', 'T16', 'T18', 'T1...","[9, 7, 5, 3, 8, 4, 2, 6, 10, 1]","[7, 9, 12, 10, 11, 13, 5, 8, 14, 4]","[6, 2, 1, 5, 9, 7, 4, 8, 10, 3]","[331, 350, 347]","[154, 154, 143]","['GS1', 'GS2', 'GS3']","[38, 39, 27]","[19, 17, 'GS3', 15, 10, 20, 18, 14, 'GS2', 13,...","['GS3', 'GS2', 20, 14, 17, 16, 13, 18, 11, 19,...","[17, 16, 18, 'GS3', 'GS2', 20, 19, 'GS1', 14]"


In [10]:
input_json_array = []
for index, row in eos_data_df.iterrows():
    input_json_array.append({'Targets': row['Targets'], 'Priorities': row['Priorities'],
                             'Size of Targets': row['Size of Targets'], 
                             'Energies of Targets': row['Energies of Targets'],
                             'Storage Limit of Satellites': row['Storage Limit of Satellites'], 
                             'Energy Limit of Satellites': row['Energy Limit of Satellites'],
                             'Ground Stations': row['Ground Stations'], 
                             'Limit of Ground Stations': row['Limit of Ground Stations'],
                             'Satellite 1': row['Satellite 1'], 'Satellite 2': row['Satellite 2'],
                             'Satellite 3': row['Satellite 3']                            
                            })
input_json = json.dumps(input_json_array)

In [16]:
print(input_json)

[{"Targets": "['T18', 'T43', 'T10', 'T26', 'T36', 'T11', 'T44', 'T20', 'T45', 'T31', 'T12', 'T15', 'T46', 'T33', 'T34', 'T38', 'T35', 'T23', 'T27', 'T40', 'T19', 'T30', 'T37', 'T13', 'T32', 'T29', 'T41', 'T25', 'T24', 'T21', 'T17', 'T22', 'T39', 'T42', 'T14', 'T16']", "Priorities": "[30, 3, 9, 26, 14, 10, 29, 27, 2, 11, 24, 18, 8, 5, 15, 25, 31, 28, 33, 20, 34, 12, 1, 23, 6, 17, 21, 19, 7, 35, 13, 32, 22, 16, 36, 4]", "Size of Targets": "[36, 20, 28, 14, 15, 16, 24, 12, 26, 18, 9, 22, 31, 13, 34, 11, 7, 35, 30, 23, 10, 37, 6, 32, 40, 27, 29, 39, 4, 19, 17, 25, 8, 38, 33, 21]", "Energies of Targets": "[26, 10, 23, 19, 9, 4, 14, 6, 11, 25, 1, 8, 2, 12, 20, 35, 24, 3, 34, 18, 36, 15, 22, 16, 21, 31, 30, 27, 28, 33, 32, 29, 5, 7, 17, 13]", "Storage Limit of Satellites": "[296, 318, 391]", "Energy Limit of Satellites": "[100, 151, 169]", "Ground Stations": "['GS3', 'GS2', 'GS1']", "Limit of Ground Stations": "[37, 38, 39]", "Satellite 1": "[27, 25, 'GS1', 'GS2', 10, 'GS3', 37, 15, 35, 14, 2

In [28]:
def optimize_eos_using_quantum(input_json):
    output_json_array = []

    for input in input_json:
        targets = [int(target.replace('T', '')) for target in ast.literal_eval(input['Targets'])]
        priorities = [priority for priority in ast.literal_eval(input['Priorities'])]
        sizes = [priority for priority in ast.literal_eval(input['Size of Targets'])]
        energies = [energy for energy in ast.literal_eval(input['Energies of Targets'])]
        ground_stations = [int(gs.replace('GS', '')) for gs in ast.literal_eval(input['Ground Stations'])]
        gs_limit = [-gs_limit for gs_limit in ast.literal_eval(input['Limit of Ground Stations'])]
        s1 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(input['Satellite 1'])],
                                    ground_stations, gs_limit)
        s2 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(input['Satellite 2'])],
                                    ground_stations, gs_limit)
        s3 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(input['Satellite 3'])],
                                    ground_stations, gs_limit)
        energy_limit = [energy_limit for energy_limit in ast.literal_eval(input['Energy Limit of Satellites'])]
        storage_limit = [storage_limit for storage_limit in ast.literal_eval(input['Storage Limit of Satellites'])]

        s1_, gs1_ = separate_gs_and_fix_non_gs_targets(s1)

        s2_, gs2_ = separate_gs_and_fix_non_gs_targets(s2)

        s3_, gs3_ = separate_gs_and_fix_non_gs_targets(s3)

        size_dict = dict(zip(targets, sizes))

        priorities_dict = dict(zip(targets, priorities))

        energies_dict = dict(zip(targets, energies))

        size1_list = [[size_dict[key] for key in sublist] for sublist in s1_]
        # print("size1_list",size1_list)

        size2_list = [[size_dict[key] for key in sublist] for sublist in s2_]
        # print("size2_list",size2_list)

        size3_list = [[size_dict[key] for key in sublist] for sublist in s3_]
        # print("size3_list",size3_list)

        priorities1_list = [[priorities_dict[key] for key in sublist] for sublist in s1_]
        # print("priorities1_list",priorities1_list)

        priorities2_list = [[priorities_dict[key] for key in sublist] for sublist in s2_]
        # print("priorities2_list",priorities2_list)

        priorities3_list = [[priorities_dict[key] for key in sublist] for sublist in s3_]
        # print("priorities3_list",priorities3_list)

        energies1_list = [[energies_dict[key] for key in sublist] for sublist in s1_]
        # print("energies1_list",energies1_list)

        energies2_list = [[energies_dict[key] for key in sublist] for sublist in s2_]
        # print("energies2_list",energies2_list)

        energies3_list = [[energies_dict[key] for key in sublist] for sublist in s3_]
        # print("energies3_list",energies3_list)

        intial_target = [s1_, s2_, s3_]
        # print("Targets = ",intial_target)

        target_dict = {}
        c = 0
        target_ = []
        for i in intial_target:
            j_ = []
            for j in i:
                k_ = []
                for k in j:
                    k_.append(c)
                    target_dict[c] = k
                    c = c + 1
                j_.append(k_)
            target_.append(j_)

        size_ = [size1_list, size2_list, size3_list]
        priority_ = [priorities1_list, priorities2_list, priorities3_list]
        energy_ = [energies1_list, energies2_list, energies3_list]

        energy_limit_ = energy_limit
        # energy limit for each satellite
        assert len(energy_) == len(energy_limit_)

        dl_ = [gs1_, gs2_, gs3_]
        dl_copy = copy.deepcopy(dl_)
        # downlink size limit in satellites

        assert len(dl_) == len(target_)
        dl_sizes = [len(i) for i in dl_]
        quadrent_sizes = [len(i) for i in target_]
        assert dl_sizes == quadrent_sizes

        gs_ = storage_limit
        # satellite storge in satellites

        assert len(gs_) == len(target_)

        name = 0

        q_targets = list(chain(*list(chain(*intial_target))))
        targets = list(chain(*list(chain(*target_))))
        sizes = list(chain(*list(chain(*size_))))
        priority = list(chain(*list(chain(*priority_))))
        energy = list(chain(*list(chain(*energy_))))

        assert len(targets) == len(sizes)
        assert len(priority) == len(targets)
        assert len(energy) == len(sizes)

        dl_ = [[-1 * j for j in i] for i in dl_]

        converted_dict = {}
        for key, value in target_dict.items():
            if value in converted_dict:
                converted_dict[value].append(key)
            else:
                converted_dict[value] = [key]

        keys = [k for k, v in converted_dict.items() if len(v) > 1]

        options_ = []
        for key in keys:
            options_.append(converted_dict[key])

        cqm = ConstrainedQuadraticModel()
        targets_included = []
        for j in range(len(priority)):
            targets_included.append(-priority[j] * Binary(f't{targets[j]}'))
        cqm.set_objective(sum(targets_included))
        target_result = [(k) * Binary(f't{j}') for j, k in zip(range(len(targets)), sizes)]

        energy_array = [(k) * Binary(f't{j}') for j, k in zip(range(len(targets)), energy)]

        for target, size, dl, gs, energy_limit in zip(target_, size_, dl_, gs_, energy_limit_):
            count = target[0][0]
            cons, dl_limits = constraints_func(dl, gs)
            cons.reverse()
            counts = []
            for i in zip(target, cons):
                cqm.add_constraint(sum(target_result[count:count + len(i[0])]) <= i[1],
                                   label=f'per q {name} {count}:{count + len(i[0])}<={i[1]}')
                name = name + 1
                counts.append(count)
                count = count + len(i[0])

            dl_limits.reverse()

            for i in zip(counts[:-1], dl_limits[:-1]):
                cqm.add_constraint(sum(target_result[i[0]:count]) <= i[1], label=f'per q {name} {i[0]}:{count}<={i[1]}')
                name = name + 1

            cqm.add_constraint(sum(energy_array[target[0][0]:count]) <= energy_limit,
                               label=f'{name} energy limit {target[0][0]}:{count} {energy_limit}')

        for i in range(len(options_)):
            cqm.add_constraint(sum([Binary(f't{j}') for j in options_[i]]) <= 1, label=f'common {i} {options_[i]}')

#         qstart = time.time()

#         sampler = LeapHybridCQMSampler(token="DEV-a837e60da7c343e9e89595965e4d264c5db6c402")
#         sampleset = sampler.sample_cqm(cqm)
#         # sampleset = ExactCQMSolver().sample_cqm(cqm)
#         feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
#         result = []
#         for i in feasible_sampleset.first.sample.items():
#             if i[1] == 1.0:
#                 result.append(int(i[0].replace("t", "")))
#         result.sort()
#         qend = time.time()
#         q_time = (qend - qstart) * 10 ** 3

#         output_json = post(result, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_,
#                            gs2_, gs3_, q_targets,
#                            sizes, energy,
#                            energy_limit_, q_time)
        cstart = time.time()

        bqm, invert = dimod.cqm_to_bqm(cqm)
        qubo = bqm.to_qubo()
        sampler = neal.SimulatedAnnealingSampler()
        sampleset = sampler.sample(bqm, seed=1234,
                               beta_range=[0.1, 4.2],
                               num_sweeps=20,
                               beta_schedule_type='geometric')
        data = sampleset.first.sample
        sa = []
        for i in targets:
            if (data[f"t{i}"]) == 1:
                sa.append(i)

        cend = time.time()
        c_time = (cend - cstart) * 10 ** 3
        
#         print('sa results: ', sa)

        simulated_out_json_inter = post_json(sa, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_,
                                        gs2_, gs3_, q_targets, sizes,
                                        energy, energy_limit_, c_time)

        output_json_array.append(simulated_out_json_inter)
    return output_json_array

In [63]:
output_json = optimize_eos_using_quantum(json.loads(input_json))
print(output_json)

sa results:  [3, 7, 8, 9, 26, 27, 30, 35, 40, 56, 63, 64, 65]
iq*****  [[[14, 16, 27, 25], []], [[35, 33, 10], [25], [22]], [[], [33], [21, 22, 29]]]
Storage total is beyond 123 114
sa results:  [7, 8, 9, 50, 52, 55, 58, 80, 95, 98, 99, 102, 128, 138, 142, 143]
iq*****  [[[], [45], [56, 50], [], [53, 57]], [[74], [80, 67], [], [49]], [[50, 80, 84], [45], [], [], [67], [82, 33]]]
sa results:  [1, 3, 7, 9, 11, 14, 15, 20, 24, 30, 33, 35]
iq*****  [[[13, 35], [24, 27, 30, 17, 10], []], [[26, 34], [16, 29, 15]], [[]]]
Storage total is beyond 110 98
sa results:  [7, 8, 9, 24, 98, 99, 147, 151]
iq*****  [[[], [94, 19, 34], [], [], [31]], [[], [], [], [], [], [53, 77], [], []], [[], [], [], [], [], [47], [], [24]]]
sa results:  [8, 9, 10, 11, 14, 24]
iq*****  [[[], [], [16]], [[15], [20, 14, 13]], [[], [19]]]
sa results:  [8, 9, 65, 66, 67, 95, 98, 99, 124, 127, 130]
iq*****  [[[], [16, 39], [], [], [], [67, 66, 47]], [[], [], [], [16, 59, 42], []], [[], [], [], [64, 58], [80]]]
sa results:  